Notebook para a divisão k-fold

In [29]:
import pandas as pd

In [30]:
import numpy as np

In [31]:
df = pd.read_csv('blogset-duplicate.csv')

In [32]:
df.shape

(2604, 5)

In [33]:
df.dropna(axis=1, inplace=True)

In [34]:
df.shape

(2604, 5)

In [35]:
df.columns

Index(['AuthorID', 'Qual sua idade?', 'Qual seu sexo?', 'Texts', 'AgeClass'], dtype='object')

In [36]:
def genderGroups(text):
    if(text == "Masculino"):
        return 0
    else:
        return 1

In [37]:
df["GenderClass"] = df["Qual seu sexo?"].apply(genderGroups)

In [38]:
df["Qual seu sexo?"].value_counts()

Masculino    1564
Feminino     1040
Name: Qual seu sexo?, dtype: int64

In [9]:
from sklearn.model_selection import StratifiedKFold

In [10]:
kf_strat = StratifiedKFold(n_splits=10, shuffle=True)

In [11]:
df.reset_index(drop=True, inplace=True)

In [16]:
def GroupAgeAndGender(df):
    ageGender = []
    for _, line in df.iterrows():
        if line["AgeClass"]==0 and line["GenderClass"]==1:
            ageGender.append(0)
        elif line["AgeClass"]==0 and line["GenderClass"]==0:
            ageGender.append(1)
        elif line["AgeClass"]==1 and line["GenderClass"]==1:
            ageGender.append(2)
        elif line["AgeClass"]==1 and line["GenderClass"]==0:
            ageGender.append(3)
        elif line["AgeClass"]==2 and line["GenderClass"]==1:
            ageGender.append(4)
        elif line["AgeClass"]==2 and line["GenderClass"]==0:
            ageGender.append(5)
    df["AgeGender"] = ageGender

In [17]:
GroupAgeAndGender(df)

In [19]:
x = df
y = df["AgeGender"]

In [20]:
def getsplits_strat(df, samples):
     kfold_list = []
     for train_index, test_index in kf_strat.split(df, samples):
          print(len(train_index), len(test_index))
          dftrain = df.iloc[train_index]
          dftest = df.iloc[test_index]
          kfold_list.append([dftrain, dftest])
     return kfold_list

In [21]:
kfold_list = getsplits_strat(x, y)

2283 254
2283 254
2283 254
2283 254
2283 254
2283 254
2283 254
2284 253
2284 253
2284 253


In [23]:
for train, test in kfold_list:
    train_fem = train[train["GenderClass"]==1]
    train_masc = train[train["GenderClass"]==0]
    print(train_fem["AgeClass"].value_counts())
    print(train_masc["AgeClass"].value_counts())

    test_fem = test[test["GenderClass"]==1]
    test_masc = test[test["GenderClass"]==0]
    print(test_fem["AgeClass"].value_counts())
    print(test_masc["AgeClass"].value_counts())

1    371
2    299
0    243
Name: AgeClass, dtype: int64
1    522
2    499
0    349
Name: AgeClass, dtype: int64
1    41
2    33
0    27
Name: AgeClass, dtype: int64
1    58
2    56
0    39
Name: AgeClass, dtype: int64
1    371
2    299
0    243
Name: AgeClass, dtype: int64
1    522
2    499
0    349
Name: AgeClass, dtype: int64
1    41
2    33
0    27
Name: AgeClass, dtype: int64
1    58
2    56
0    39
Name: AgeClass, dtype: int64
1    371
2    299
0    243
Name: AgeClass, dtype: int64
1    522
2    499
0    349
Name: AgeClass, dtype: int64
1    41
2    33
0    27
Name: AgeClass, dtype: int64
1    58
2    56
0    39
Name: AgeClass, dtype: int64
1    370
2    299
0    243
Name: AgeClass, dtype: int64
1    522
2    500
0    349
Name: AgeClass, dtype: int64
1    42
2    33
0    27
Name: AgeClass, dtype: int64
1    58
2    55
0    39
Name: AgeClass, dtype: int64
1    370
2    299
0    243
Name: AgeClass, dtype: int64
1    522
2    500
0    349
Name: AgeClass, dtype: int64
1    42
2    33


### Salvar em arquivo

In [25]:
def create_split_csv(l, path):
    import os
    import shutil

    root_folder = path
    folder_name = "k_"
    file_name1 = "train"
    file_name2 = "test"
    k = 1

    for training, test in l:
        path = os.path.join(root_folder, (folder_name+str(k)))
        if(os.path.exists(path)):
            shutil.rmtree(path)
        os.mkdir(path)

        training.to_csv((path+"/"+file_name1+".csv"), index = False, header=True)
        test.to_csv((path+"/"+file_name2+".csv"), index = False, header=True)

        k+=1

In [26]:
create_split_csv(kfold_list, "C:/Users/alici/Documents/tcc/github2/tcc-v2/blogset-br/particoes")